In [1]:
import sqlite3
import os
import glob

from textwrap import dedent
from dotenv import load_dotenv

import pandas as pd
from crewai import LLM
from crewai import Agent, Crew, Process, Task
from crewai.tools import tool
from crewai_tools import MCPServerAdapter
from langchain.schema import AgentFinish
from langchain.schema.output import LLMResult
from langchain_community.tools.sql_database.tool import (
    InfoSQLDatabaseTool,
    ListSQLDatabaseTool,
    QuerySQLCheckerTool,
    QuerySQLDataBaseTool,
)
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_core.callbacks.base import BaseCallbackHandler
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

load_dotenv()

c:\Users\USER\anaconda3\envs\crewai-py311\Lib\site-packages\pydantic\_internal\_config.py:323: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)


True

In [20]:
conn = sqlite3.connect("tcmbank_database.db")

# Get all CSV files in a folder
csv_files = glob.glob(os.path.join(".", "data", "raw", "tcm_bank_csv", "*.csv"))
print(csv_files)

# Loop through CSV files and import each into SQLite
for file in csv_files:
    # Use filename (without extension) as table name
    table_name = file.split("/")[-1].replace(".csv", "")

    # Read CSV into DataFrame, trying 'latin-1' encoding
    try:
        df = pd.read_csv(file, encoding='latin-1')
    except UnicodeDecodeError:
        # If 'latin-1' fails, try another common encoding like 'cp1252'
        try:
            df = pd.read_csv(file, encoding='cp1252')
        except Exception as e:
            print(f"Could not read file {file} with latin-1 or cp1252 encoding: {e}")
            continue # Skip to the next file

    # Write to SQLite (if table exists, replace it)
    df.to_sql(table_name, conn, if_exists="replace", index=False)

print("All CSV files have been imported into SQLite!")
conn.close()

[]
All CSV files have been imported into SQLite!


In [3]:
llm = ChatGoogleGenerativeAI(model="gemini-2.5-pro")
agent= LLM(model="gemini/gemini-2.5-pro")

In [4]:
db = SQLDatabase.from_uri("sqlite:///tcmbank_database.db")

In [6]:
@tool("list_tables")
def list_tables() -> str:
  """List the available tables in the database"""
  return ListSQLDatabaseTool(db=db).invoke("")

list_tables.run()

Using Tool: list_tables


''

In [6]:
@tool("tables_schema")
def tables_schema(tables: str) -> str:
  """
  Input is a comma-separated list of tables, output is the schema and sample rows for those
  tables. Be sure that the tables actually exist before calling `list_tables` first!
  Example Input: table1, table2, table3
  """
  tool = InfoSQLDatabaseTool(db=db)
  return tool.invoke(tables)

print(tables_schema.run("herb_all"))

Using Tool: tables_schema

CREATE TABLE herb_all (
	"TCMBank_ID" TEXT, 
	level1_name_en TEXT, 
	level2_name TEXT, 
	"TCM_name" TEXT, 
	"TCM_name_en" TEXT, 
	"Herb_pinyin_name" TEXT, 
	"Herb_latin_name" TEXT, 
	"Properties" TEXT, 
	"Meridians" TEXT, 
	"UsePart" TEXT, 
	"Function" TEXT, 
	"Indication" TEXT, 
	"Toxicity" TEXT, 
	"Clinical_manifestations" TEXT, 
	"Therapeutic_en_class" TEXT, 
	"Therapeutic_cn_class" TEXT, 
	"TCMID_id" TEXT, 
	"TCM_ID_id" TEXT, 
	"SymMap_id" TEXT, 
	"TCMSP_id" TEXT, 
	"Herb_ID" TEXT
)

/*
3 rows from herb_all table:
TCMBank_ID	level1_name_en	level2_name	TCM_name	TCM_name_en	Herb_pinyin_name	Herb_latin_name	Properties	Meridians	UsePart	Function	Indication	Toxicity	Clinical_manifestations	Therapeutic_en_class	Therapeutic_cn_class	TCMID_id	TCM_ID_id	SymMap_id	TCMSP_id	Herb_ID
TCMBANKHE000001	None	None	???	Turpentine Oil	SONG JIE YOU	Oleum Terebinthinae	None	None	None	Treatment of myalgia, arthralgia	Treatment of myalgia, arthralgia	None	None	None	None	None	210

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [7]:
@tool("execute_sql")
def execute_sql(sql_query: str) -> str:
  """Execute a SQL query against the database. Returns the result"""
  return QuerySQLDataBaseTool(db=db).invoke(sql_query)

execute_sql.run("SELECT * FROM herb_all WHERE TCM_name_en LIKE '%Turpentine Oil%'")
# execute_sql.run("SELECT DISTINCT TCM_name_en FROM herb_all")

Using Tool: execute_sql


/tmp/ipython-input-2267673259.py:4: LangChainDeprecationWarning: The class `QuerySQLDataBaseTool` was deprecated in LangChain 0.3.12 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-community package and should be used instead. To use it run `pip install -U :class:`~langchain-community` and import as `from :class:`~langchain_community.tools import QuerySQLDatabaseTool``.
  return QuerySQLDataBaseTool(db=db).invoke(sql_query)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


"[('TCMBANKHE000001', None, None, '???', 'Turpentine Oil', 'SONG JIE YOU', 'Oleum Terebinthinae', None, None, None, 'Treatment of myalgia, arthralgia', 'Treatment of myalgia, arthralgia', None, None, None, None, None, '2107', None, '1047', 'HERB005260')]"

In [10]:
@tool("check_sql")
def check_sql(sql_query: str) -> str:
  """
  Use this tool to double check if your query is correct before executing it. Always use this tool before
  using executing a query with `execute_sql`.
  """
  # Use the LangChain ChatGoogleGenerativeAI instance instead of the crewai.LLM wrapper
  return QuerySQLCheckerTool(db=db, llm=llm).invoke({"query": sql_query})

check_sql.run("SELECT DITINCT TCM_name_en FRO herb_all")

Using Tool: check_sql


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


'```sql\nSELECT DISTINCT TCM_name_en FROM herb_all\n```'

### Creating Agents

In [11]:
sql_dev = Agent(
    role="Senior Database Developer",
    goal="Construct and execute SQL queries based on a request",
    backstory=dedent(
        """
        You are an experienced database engineer who is master at creating efficient and complex SQL queries.
        You have a deep understanding of how different databases work and how to optimize queries.
        You have access to the following table descriptions to help you understand the database schema:

        *   **disease_all:** This table contains information about various diseases, including their names and potentially related Traditional Chinese Medicine (TCM) concepts. Key columns might include disease names in English and Chinese, and possibly links to other relevant data.<br>**Use For**: Disease identification, medical classifications, symptom mapping, disease relationships.
        *   **gene_all:** This table contains information about genes/proteins that serve as therapeutic targets with chromosomal locations and target validation status. Key columns might include gene identifiers and names.<br>**Use For**: Disease identification, medical classifications, symptom mapping, disease relationships.
        *   **herb_all:** This table contains information about traditional Chinese herbs. It includes details such as TCM names, English names, Latin names, properties, meridians, functions, and indications.<br>**Use For**: Traditional TCM knowledge, herb properties, meridian theory, classical indications
        *   **ingredient_all:** This table contains information about active chemical ingredients/compounds found in TCM herbs. It includes various identifiers, chemical properties, and potentially links to herbs or formulas.<br>**Use For**: Chemical structures, drug-likeness, pharmacokinetic properties, molecular identification

        ## Key Relationships & Data Flow

        **Primary Research Chain**: Herbs → Ingredients → Genes → Diseases

        ### Critical Connections:
        1. **TCMBank_ID**: Links herbs to their constituent ingredients
        2. **Source_ID**: Cross-references across external databases
        3. **Traditional-Modern Bridge**: Herb indications ↔ Disease classifications
        4. **Molecular Mechanisms**: Ingredient structures ↔ Gene targets ↔ Disease pathways

        ## Retrieval Instructions

        ### Query Processing Strategy:
        1. **Identify Query Type**:
          - Traditional TCM (herb names, properties, meridians)
          - Chemical/Molecular (compound names, structures, targets)
          - Medical (diseases, symptoms, conditions)
          - Mechanistic (how/why questions about TCM effects)

        2. **Multi-File Search Approach**:
          - **Single Entity Queries**: Start with the most relevant file, then expand
          - **Relationship Queries**: Search across multiple files simultaneously
          - **Mechanism Queries**: Follow the Herb→Ingredient→Gene→Disease pathway

        3. **Search Field Prioritization**:
          - **Primary**: Exact name matches, IDs
          - **Secondary**: Aliases, synonyms, alternative names
          - **Tertiary**: Descriptions, functions, classifications
          - **Contextual**: Related terms, broader categories

        ### Specific Retrieval Patterns:

        **For Traditional TCM Queries**:
        - Search herb_all for: TCM_name, Properties, Meridians, Function, Indication
        - Cross-reference with ingredient_all using TCMBank_ID
        - Link to disease_all through indication matching

        **For Chemical/Pharmacological Queries**:
        - Search ingredient_all for: compound names, molecular properties, ADMET data
        - Connect to gene_all for target information
        - Trace back to herb_all for source herbs

        **For Disease/Medical Queries**:
        - Search disease_all using multiple classification systems
        - Find related genes in gene_all
        - Identify targeting ingredients in ingredient_all
        - Trace to source herbs in herb_all

        **For Mechanism/Integration Queries**:
        - Follow complete pathway: specific herb → active ingredients → molecular targets → disease effects
        - Use cross-references and IDs to maintain data integrity
        - Combine traditional knowledge with molecular evidence


        Use the `list_tables` to find available tables.
        Use the `tables_schema` to understand the metadata for the tables.
        Use the `check_sql` to check your queries for correctness.
        Use the `execute_sql` to execute queries against the database.

    """
    ),
    llm=agent,
    tools=[list_tables, tables_schema, execute_sql, check_sql],
    allow_delegation=False,
)

In [12]:
data_analyst = Agent(
    role="Senior Data Analyst",
    goal="You receive data from the database developer and analyze it",
    backstory=dedent(
        """
        You have deep experience with analyzing datasets using Python.
        Your work is always based on the provided data and is clear,
        easy-to-understand and to the point. You have attention
        to detail and always produce very detailed work (as long as you need).
    """
    ),
    llm=agent,
    allow_delegation=False,
)

In [13]:
report_writer = Agent(
    role="Senior Report Editor",
    goal="Write an executive summary type of report based on the work of the analyst",
    backstory=dedent(
        """
        Your writing skill is well known for clear and effective communication.
        You always summarize long texts into briefs that contain the most
        important details.
        """
    ),
    llm=agent,
    allow_delegation=False,
)

In [17]:
# Basic API setup (port: 8080, path: /mcp)
# !SERVER_TRANSPORT=streamable-http SERVER_PORT=3001 npx @sei-js/mcp-server

# Point CrewAI to your SEI MCP server
server_params = {
    "url": "http://localhost:8080/mcp"   # matches the SEI server
}

# Managed connection context (auto start/stop)
with MCPServerAdapter(server_params, connect_timeout=60) as tools:
    print("Available tools from SEI MCP server:", [t.name for t in tools])

    # Create an agent that can use the SEI MCP tools
    sei_agent = Agent(
        role="SEI Integrator",
        goal="Use SEI MCP tools to query blockchain or perform tasks.",
        backstory="Specialist agent that integrates with SEI MCP server.",
        tools=tools,
        verbose=True
    )

    # Create a task that tells CrewAI which tool to use
    sei_task = Task(
        description=f"Use `{tools[0].name}` to fetch some data from SEI.",  # pick an actual tool name
        expected_output="Response from the SEI MCP tool.",
        agent=sei_agent,
    )

    crew = Crew(
        agents=[sei_agent],
        tasks=[sei_task],
        process=Process.sequential,
        verbose=True
    )

    result = crew.kickoff()
    print("Crew Task Result:", result)



Exception in thread Thread-11 (_run_loop):
  + Exception Group Traceback (most recent call last):
  |   File "/usr/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
  |     self.run()
  |   File "/usr/lib/python3.12/threading.py", line 1012, in run
  |     self._target(*self._args, **self._kwargs)
  |   File "/usr/local/lib/python3.12/dist-packages/mcpadapt/core.py", line 237, in _run_loop
  |     self.loop.run_until_complete(self.task)
  |   File "/usr/lib/python3.12/asyncio/base_events.py", line 691, in run_until_complete
  |     return future.result()
  |            ^^^^^^^^^^^^^^^
  |   File "/usr/local/lib/python3.12/dist-packages/mcpadapt/core.py", line 226, in setup
  |     await stack.enter_async_context(
  |   File "/usr/lib/python3.12/contextlib.py", line 659, in enter_async_context
  |     result = await _enter(cm)
  |              ^^^^^^^^^^^^^^^^
  |   File "/usr/lib/python3.12/contextlib.py", line 210, in __aenter__
  |     return await anext(self.gen)
  |     

Error connecting to or using Streamable HTTP MCP server (Managed): Failed to initialize MCP Adapter: Couldn't connect to the MCP server after 30 seconds
Ensure the Streamable HTTP MCP server is running and accessible at the specified URL.


### Create Tasks

In [ ]:
extract_data = Task(
    description="Extract data that is required for the query {query}.",
    expected_output="Database result for the query",
    agent=sql_dev,
)

In [ ]:
analyze_data = Task(
    description="Analyze the data from the database and write an analysis for {query}.",
    expected_output="Detailed analysis text",
    agent=data_analyst,
    context=[extract_data],
)

In [ ]:
write_report = Task(
    description=dedent(
        """
        Write an executive summary of the report from the analysis. The report
        must be less than 100 words.
    """
    ),
    expected_output="Markdown report",
    agent=report_writer,
    context=[analyze_data],
)

In [ ]:
crew = Crew(
    agents=[sql_dev, data_analyst, report_writer],
    tasks=[extract_data, analyze_data, write_report],
    process=Process.sequential,
    verbose=True,
    memory=False,
)

In [ ]:
inputs = {
    "query": "What is turpentine oil used for?"
}

result = crew.kickoff(inputs=inputs)


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 48fbb3e3-f41d-486d-b0ae-53c0bc8c5702                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Database Developer                                                                               │
│                                                                                                                 │
│  Task: Extract data that is required for the query What is turpentine oil used for?.                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

/usr/local/lib/python3.12/dist-packages/ipywidgets/widgets/widget_output.py:111: DeprecationWarning: 
Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  if ip and hasattr(ip, 'kernel') and hasattr(ip.kernel, '_parent_header'):
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: 
datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects 
to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)

...

/usr/local/lib/python3.12/dist-packages/ipywidgets/widgets/widget_output.py:112: DeprecationWarning: 
Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  self.msg_id = ip.kernel._parent_header['header']['msg_id']

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


╭───────────────────────────────────────────────── Task Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Failed                                                                                                    │
│  Name: 4081f05f-7c6d-478b-b03b-30c3b6fcd0ef                                                                     │
│  Agent: Senior Report Editor                                                                                    │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Crew Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Failed                                                                                          │
│  Name: crew                                                                                                     │
│  ID: 48fbb3e3-f41d-486d-b0ae-53c0bc8c5702                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output:                                                                                                  │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

ValueError: Invalid response from LLM call - None or empty.